# Post-procesamiento

Ahora que ya hemos realizado una predicción sobre imágenes con nuestro modelo pre-entrenado, podemos trabajar sobre los resultados para llegar a datos más reutilizables.

El objetivo es obtener un archivo vectorial (Shapefile, GeoJSON, etc.) de polígonos o multipolígonos, de las áreas detectadas como asentamientos informales. Para esto realizaremos los siguientes pasos:

1. Suavizado
2. Filtro por umbral
2. Poligonización
3. Filtro de área mínima

## Preparación de datos

A efectos de este webinar, nos descargamos primero los resultados de la predicción de la sesión anterior:

In [1]:
results_dir = "../data/results"

In [2]:
!wget https://storage.googleapis.com/dym-workshops-public/aplatam-net/session3/honduras_predict_results.zip -O results.zip
!mkdir -p $results_dir
!unzip -o -d $results_dir results.zip

--2022-07-16 20:02:55--  https://storage.googleapis.com/dym-workshops-public/aplatam-net/session3/honduras_predict_results.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.133.16, 142.251.133.80, 172.217.172.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.133.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1096065 (1,0M) [application/zip]
Saving to: ‘results.zip’

results.zip         100%[===================>]   1,04M  --.-KB/s    in 0,1s    

2022-07-16 20:02:55 (8,40 MB/s) - ‘results.zip’ saved [1096065/1096065]

Archive:  results.zip
  inflating: ../data/results/honduras_0000000000-0000000000_0_22.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_0_23.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_0_24.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_10_20.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_10_21.tif  
  inflating: ../

  inflating: ../data/results/honduras_0000000000-0000000000_22_17.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_22_18.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_22_19.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_22_20.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_22_31.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_22_32.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_17.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_18.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_19.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_20.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_26.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_27.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_23_28.tif  
  inflating: ../data/results/honduras_0000000000-0000000000_24_1

  inflating: ../data/results/honduras_0000014848-0000000000_24_68.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_24_69.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_24_70.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_24_71.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_24_72.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_24_73.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_24_74.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_67.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_68.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_69.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_70.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_71.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_72.tif  
  inflating: ../data/results/honduras_0000014848-0000000000_25_7

## Filtro por umbral

En este paso se filtran las imágenes aplicando un umbral sobre los valores de los píxeles de cada ráster, que en este caso representan la probabilidad (valores entre 0 y 1), quedándonos así con las de mayor precisión.

*Nota*: En realidad `unetseg` al predecir genera rásteres con valores entre 0 y 255, porque utiliza el tipo de datos Byte, que es de 8 bits. Para convertir estos valores entre 0 y 255 a 0.0 y 1.0, simplemente se divide el numero por 255. La razón de esta optimización es porque almacenar números de punto flotante es más costoso que almacenar en bytes (al menos 4 u 8 veces más costoso en espacio).

In [3]:
from satproc.filter import filter_by_max_prob

In [4]:
filtered_results_dir = "../data/filtered"
threshold = 0.5

In [5]:
!rm -rf $filtered_results_dir

In [6]:
filter_by_max_prob(input_dir=results_dir,
                   output_dir=filtered_results_dir,
                   threshold=threshold)

100%|##############################################################################################################################################################| 457/457 [00:01<00:00, 415.01it/s]


## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina utiliza `gdal_polygonize.py` sobre cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo y disuelve las geometrías, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1). Utilizaremos el mismo umbral que usamos en el paso anterior de filtrado.

In [7]:
from satproc.postprocess.polygonize import polygonize

In [8]:
polygonized_results_path = "../data/polygonized.gpkg"

In [9]:
polygonize(threshold=threshold,
           input_dir=filtered_results_dir,
           output=polygonized_results_path)

Dissolve: 100%|#########################################################################################################################################################| 5/5 [00:01<00:00,  2.63it/s]


In [10]:
import folium
import geopandas as gpd

In [11]:
geojson_path = "../data/polygonized.geojson"

gpd.read_file(polygonized_results_path).to_file(geojson_path, driver="GeoJSON")

/home/munshkr/.local/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [12]:
m = folium.Map(location=[14.073297, -87.202824], zoom_start=12)
folium.GeoJson(geojson_path, name="geojson").add_to(m)
m

Se puede observar que hay pequeños polígonos causados por falsos positivos que podrían ser eliminados.

## Filtrar por área mínima

Para resolver este problema, como último paso, filtramos aquellos polígonos de area inferior a 3000 m². Para esto utilizamos `ogr2ogr` y una consulta SQL.  Antes de aplicar el filtro, debemos reproyectar el GeoPackage a un CRS proyectado como UTM.

In [13]:
utm_polygonized_path = "../data/polygonized_utm.gpkg"

!ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32616 -f 'GPKG' $utm_polygonized_path $polygonized_results_path

In [14]:
output_path = "../data/output.gpkg"
min_area = 3000

!ogr2ogr \
    -t_srs EPSG:4326 \
    -f "GPKG" \
    -sql "SELECT * FROM polygonized m WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $output_path \
    $utm_polygonized_path

Resultado final:

In [15]:
geojson_path = "../data/output.geojson"
gpd.read_file(output_path).to_file(geojson_path, driver="GeoJSON")

/home/munshkr/.local/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [16]:
m = folium.Map(location=[14.073297, -87.202824], zoom_start=12)
folium.GeoJson(geojson_path, name="geojson").add_to(m)
m